In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import itertools
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm_notebook

C:\Users\Hubola\AppData\Local\Temp\ipykernel_11092\731219046.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv')

In [3]:
data.head()

crim    zn  indus  chas    nox     rm   age     dis  rad  tax  ptratio  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296     15.3   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242     17.8   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242     17.8   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222     18.7   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222     18.7   

        b  lstat  medv  
0  396.90   4.98  24.0  
1  396.90   9.14  21.6  
2  392.83   4.03  34.7  
3  394.63   2.94  33.4  
4  396.90   5.33  36.2

In [4]:
data.shape

(506, 14)

In [5]:
# Separate features (X) and target variable (y)
X = data.drop('medv', axis=1)
y = data['medv']

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
tensor_X_train = torch.tensor(X_train, dtype=torch.float32)
tensor_y_train = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)
tensor_X_test = torch.tensor(X_test, dtype=torch.float32)
tensor_y_test = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

In [6]:
# Define the neural network
model = nn.Sequential(
    nn.Linear(X.shape[1], 150),
    nn.ReLU(),
    nn.Dropout(0.33),
    nn.Linear(150, 1),
    nn.ReLU()
)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training the model
epochs = 1000

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(tensor_X_train)
    loss = criterion(outputs, tensor_y_train)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Evaluation on the test set
model.eval()
with torch.no_grad():
    test_outputs = model(tensor_X_test)
    test_loss = criterion(test_outputs, tensor_y_test)

print(f'\nTest Loss: {test_loss.item():.4f}')

# Convert predictions back to numpy array for evaluation
predictions = test_outputs.numpy()

Epoch [100/1000], Loss: 30.3906
Epoch [200/1000], Loss: 22.9482
Epoch [300/1000], Loss: 20.7017
Epoch [400/1000], Loss: 18.3667
Epoch [500/1000], Loss: 16.8832
Epoch [600/1000], Loss: 18.2727
Epoch [700/1000], Loss: 16.3082
Epoch [800/1000], Loss: 17.0003
Epoch [900/1000], Loss: 15.3498
Epoch [1000/1000], Loss: 12.7870

Test Loss: 12.7737
